In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [0]:
from PIL import Image
import numpy as np
import pandas as pd
from time import time
import keras
import os
from keras.applications import MobileNetV2
from keras import applications
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import np_utils


import random
import gc

import cv2    
import matplotlib.pyplot as plt
import seaborn as sns

Using TensorFlow backend.


In [0]:
df_attr = pd.read_csv('/gdrive/My Drive/flower_data/attr_kop.csv')
df_attr.set_index('image_id', inplace=True)
df_attr.shape
images_folder ='/gdrive/My\ Drive/flower_data/train'

In [0]:
import zipfile
zip_ref = zipfile.ZipFile('/gdrive/My Drive/flowers_new.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [0]:
def load_reshape_img(fname):
  
    img = load_img(fname)  
    x = img_to_array(img)/255.
    x = cv2.resize(x,(IMG_WIDTH,IMG_HEIGHT),3)    
    x = x.reshape((1,) + x.shape)
    
    return x

In [0]:
model = MobileNetV2(alpha=0.35,weights='imagenet',include_top=False,input_shape=(224, 224, 3))

W0709 12:35:39.207125 140361874618240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0709 12:35:39.223156 140361874618240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0709 12:35:39.229279 140361874618240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0709 12:35:39.250903 140361874618240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0709 12:35:39.251717 1403618746

In [0]:
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(512, activation="relu")(x)
predictions = Dense(102, activation="softmax")(x)

W0709 12:35:46.999506 140361874618240 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model2 = Model(inputs=model.input, outputs=predictions)

#Lock initial layers to not be trained
#for layer in model.layers[:52]:
  # layer.trainable = False
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 16) 432         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 16) 64          Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [0]:
DATADIR = '/content/flower_data/train'
IMG_SIZE=224
CATEGORIES = []
for i in range(1,103):
  CATEGORIES.append(str(i))

for category in CATEGORIES:  # do dogs and cats
    path = os.path.join(DATADIR,category)  # create path to dogs and cats
    for img in os.listdir(path):  # iterate over each image per dogs and cats
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_COLOR)  # convert to array

        break  # we just want one for now so break
    break  #...and one more!

In [0]:
from tqdm import tqdm
training_data = []

def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
          
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_COLOR)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            

create_training_data()

print(len(training_data))

100%|██████████| 36/36 [00:00<00:00, 273.01it/s]

6552


In [0]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 3))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X/255.0

[[[[ 54  99  83]
   [ 59 118  94]
   [ 60 126  91]
   ...
   [ 62 119 141]
   [ 36  86  95]
   [ 28  73  78]]

  [[ 67 116  94]
   [ 79 142 113]
   [ 44 111  72]
   ...
   [ 51 106 127]
   [ 29  77  80]
   [ 19  63  63]]

  [[ 86 139 111]
   [106 172 138]
   [ 42 108  67]
   ...
   [ 33  83 105]
   [ 24  64  69]
   [ 26  64  65]]

  ...

  [[ 42  54  42]
   [ 43  53  41]
   [ 39  51  38]
   ...
   [ 43  60  46]
   [ 40  59  42]
   [ 38  55  39]]

  [[ 40  51  35]
   [ 39  44  29]
   [ 40  44  28]
   ...
   [ 32  50  33]
   [ 36  58  40]
   [ 37  59  41]]

  [[ 32  44  26]
   [ 36  40  24]
   [ 41  42  26]
   ...
   [ 32  50  33]
   [ 34  56  38]
   [ 31  53  35]]]]


In [0]:
model2.compile(optimizer=SGD(lr=0.0001, momentum=0.9)
                    , loss='sparse_categorical_crossentropy'
                    , metrics=['accuracy'])





W0709 12:36:18.520871 140361874618240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
model2.fit(X, y, batch_size=8, epochs=60, validation_split=0.1)

Train on 5896 samples, validate on 656 samples
Epoch 1/60
5896/5896 [==============================] - 47s 8ms/step - loss: 2.6692 - acc: 0.4115 - val_loss: 7.6380 - val_acc: 0.0000e+00
Epoch 2/60
5896/5896 [==============================] - 47s 8ms/step - loss: 2.0065 - acc: 0.5577 - val_loss: 8.4668 - val_acc: 0.0061
Epoch 3/60
5896/5896 [==============================] - 47s 8ms/step - loss: 1.5483 - acc: 0.6420 - val_loss: 9.3630 - val_acc: 0.0183
Epoch 4/60
5896/5896 [==============================] - 47s 8ms/step - loss: 1.2309 - acc: 0.7042 - val_loss: 10.0392 - val_acc: 0.0244
Epoch 5/60
5896/5896 [==============================] - 47s 8ms/step - loss: 0.9894 - acc: 0.7626 - val_loss: 10.4867 - val_acc: 0.0244
Epoch 6/60
5896/5896 [==============================] - 47s 8ms/step - loss: 0.8345 - acc: 0.7973 - val_loss: 11.0469 - val_acc: 0.0259
Epoch 7/60
5896/5896 [==============================] - 47s 8ms/step - loss: 0.7117 - acc: 0.8260 - val_loss: 11.3677 - val_acc: 0.0259


In [0]:
def load_reshape_img(fname): 

  img = load_img(fname) 
  x = img_to_array(img)/255. 
  x = cv2.resize(x,(224,224),3) 
  x = x.reshape((1,) + x.shape) 

  return x

In [0]:

res = model2.predict([load_reshape_img('/gdrive/My Drive/magn2.jpg')])
top = sorted(res[0], reverse=True)[0:3]
top_index = []
for i in top:
  top_index.append((res[0].tolist()).index(i)+1)
print(top)
print(top_index)
#np.argmax(model2.predict([load_reshape_img('/gdrive/My Drive/magn.jpg')]))+1 
#model2.predict([load_reshape_img('/gdrive/My Drive/data/rose.jpg')])

[0.41363892, 0.15588291, 0.14880723]
[87, 82, 78]


In [0]:
#model2.save('/content/my_model.h5')
new_model = keras.models.load_model('/content/my_model.h5')
new_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 16) 432         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 16) 64          Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [0]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model_file( 'my_model.h5' ) 
model = converter.convert()
file = open( 'model.lite' , 'wb' ) 
file.write( model )

9134968

In [0]:
f = open('labes.txt', 'w')
for x in range(1,103):
  f.write(str(x))
f.close()